In [102]:
import numpy as np
##import Kosarajus_alg as Kosarajus_alg

In [103]:
def normalize(M):
    """
    Subfunction for T. It normalizes the matrix given as input.
    
    INPUT:
        - M = A matrix M.
        
    OUTPUT:
        - M0 = The matrix M normalized, with rows that add to 1.     
    """
    
    M0=np.array(M)
    if M0.ndim == 1:
        s= M0.sum()
        return np.divide(M0,s)
        
    elif M0.ndim == 2:
        s=M0.sum(axis=1)
        return np.divide(M0,s[:,np.newaxis])
    else:
        return "Normalize. Wrong input"

In [119]:
def stationary_sol(T):
    """
    Given a random matrix T, returns a stationary sol a, s.t. aT=a
    
    INPUT:
        - T = Some square matrix.
        
    OUTPUT:
        - the stationary vector of the matrix.   
    """
    
    T0=normalize(T)
    eig=np.linalg.eig(T0)
    for i in range(len(eig[0])):
        if np.around(eig[0][i])==1: #considers only integer part
            return np.real(eig[1][i])
    return "No eigenvalue 1"

In [105]:
def obtain_active_set(T):
    """
    Function for other parts of the project. It gets the biggest connected component of the matrix 
    that we are given.
    
    INPUT:
        - T = The probability transition matrix of the markov model.
    
    OUTPUT:
        - C = A square matrix. The biggest connected component of the matrix.
        - L = A list of vertices. The states of T that correspond to the biggest component.
    
    """
    
    b=0
    j=0
    components=kosarajus_algo2(T)
    for i in components:
        a=len(components[i])
        if a>b:
            b=a
            j=i
    L=list(components[j])
    L=np.sort(L)
    C=np.array([T[i,:] for i in L])
    C=np.array([C[:,i] for i in L])
    C=np.transpose(C)
    return (C,L)

In [106]:
def Assign2(u,root,LIST,components,M):
    """
    Recursive subfunction for kosarajus
    Strong components are to be represented by appointing a separate root vertex for each component,
    and assigning to each vertex the root vertex of its component.
    
    INPUT:
        
        - u = An integer, which represents a vertex (in our numeration) that has to be
        assigned to some component.
        - root = An integer, which represents a component.
        
        - LIST = A list of vertices that are not yet introduced in the dictionary.
        
        - components = A dictionary containing the vertices (numerated from 0 to n-1), 
        each vertex associated to the root representing its component.
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
    
        - It just changes the dictionary components, assigning to each vertex its root.
    
    """
    
    in_=[i for i in M[:,u]]
    
    if u in LIST:
        
        if not root in components:
            components[root]=[u]
        elif root in components:
            components[root].append(u)
        LIST.remove(u)
            
        for i in range(len(in_)):
            if not(in_[i]==0):
                Assign2(i,root,LIST,components,M)
    return

def Visit(u,Visited,L,M):
    """
    Recursive subfunction for kosarajus
    
    INPUT:
        
        - u = An integer, which represents a vertex (in our numeration).
        - Visited = A list of the vertices already visited.
        - L = an ordered list of graph vertices, that will grow to contain each vertex once.
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
        
        - It just adds in order vertices to the list L.
    
    """
    out=M[u,:]
    if not(u in Visited):
        Visited.append(u)
        for i in range(len(out)):
            if not(out[i]==0):
                Visit(i,Visited,L,M)
        L.insert(0,u) 
    return

def kosarajus_algo2(M):
    """
    Returns a dictionary containing the vertices and their inclusion in strong components.
    Strong components are to be represented by appointing a separate root vertex for each component,
    and assigning to each root the list of vertices inside that component.
    If the graph is represented as an adjacency matrix, the algorithm requires Ο(V^2) time.
    
    INPUT:
    
        - M = A transition matrix (which is the adjacency matrix of a graph).
    
    OUTPUT:
    
        - components = A dictionary containing the components (numerated from 0 to ..), 
        each root associated to a list of vertices that are part of that component.
    
    """
    
    Visited=[]
    L=[]
    
    components={}
    
    Vertices= [i for i in range(len(M[:,1]))]
    LIST=list(Vertices)
    
    for i in Vertices:
        Visit(i,Visited,L,M)
    for u in L:
        Assign2(u,u,LIST,components,M)
    return components  

In [108]:
def T_non_reversible(C):
    """
    Function to get the transition matrix from the count matrix. It simply normalizes the count matrix.
    Is easy, and useful for very large amount of data.
    
    INPUT:
        - C = Count matrix.
    
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    return normalize(C)

In [123]:
def T_reversible(C,max_iterations=100,error=0.1):
    """
    Function to get the transition matrix from the count matrix. It gives a matrix that is reversible.
    That is, the markov model obtained is reversible (it satisfies the detailed balance equations).
    Detailed balance implies that, around any closed cycle of states, there is no net flow of probability. 
    For example, it implies that, for all a, b and c,
    T( a , b ) T( b , c ) T( c , a ) = T( a , c ) T( c , b ) T( b , a ).
    
    INPUT:
        - C = Count matrix constructed with lag tau.
        - max_iterations = maximum number of iterations we allow.
        - error = error that we consider to understand that the iteration has converged.
        
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    C_i = C.sum(axis=1) #array of the sums of the rows of C
    C_j = C.sum(axis=0) #array of the sums of the columns of C
    
    P = T_non_reversible(C)
    P = (obtain_active_set(P))[0]
    pi = stationary_sol(P)
    
    P=np.multiply(P,pi)
    X_0= P #initial state
    
    it=0
    Er=0.2 #TO BE CHANGED
    
    while (Er > error) and (it< max_iterations):
        Xi_0= X_0.sum(axis=1) #array of the sums of the rows of X_0
        Xj_0= X_0.sum(axis=0) #array of the sums of the rows of X_0
        
        X_1= C + np.matrix.transpose(C)
        X_1=np.divide(X_1,((C_i)/(Xi_0) + (C_j)/(Xj_0)))
        
        X_0 = X_1
        it+=1
        
    P = normalize(X_1)
    return P

In [125]:
count_matrix=np.array([[ 0, 1, 0],[0,1,1],[ 1, 0,0]])
count_matrix

array([[0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [116]:
T_non_reversible(count_matrix)

array([[ 0. ,  1. ,  0. ],
       [ 0. ,  0.5,  0.5],
       [ 1. ,  0. ,  0. ]])

In [124]:
T_reversible(count_matrix)

array([[ 0.  ,  0.5 ,  0.5 ],
       [ 0.25,  0.5 ,  0.25],
       [ 0.5 ,  0.5 ,  0.  ]])

In [129]:
np.linalg.eig(T_non_reversible(count_matrix))

(array([-0.25+0.66143783j, -0.25-0.66143783j,  1.00+0.j        ]),
 array([[-0.18898224+0.5j , -0.18898224-0.5j , -0.57735027+0.j  ],
        [-0.28347335-0.25j, -0.28347335+0.25j, -0.57735027+0.j  ],
        [ 0.75592895+0.j  ,  0.75592895-0.j  , -0.57735027+0.j  ]]))

In [128]:
np.linalg.eig(T_reversible(count_matrix))

(array([  1.00000000e+00,  -5.00000000e-01,  -7.67000774e-17]),
 array([[  5.77350269e-01,   7.07106781e-01,   5.77350269e-01],
        [  5.77350269e-01,   1.61436982e-16,  -5.77350269e-01],
        [  5.77350269e-01,  -7.07106781e-01,   5.77350269e-01]]))

In [130]:
stationary_sol(T_non_reversible(count_matrix))

array([ 0.75592895,  0.75592895, -0.57735027])

In [131]:
stationary_sol(T_reversible(count_matrix))

array([ 0.57735027,  0.70710678,  0.57735027])